# Model Training with PyTorch Lightning (Beta)

> This document is currently in a minimal version without sufficient annotations. We will update it in the future.

> Author: Tianyu Du

In [1]:
from time import time
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

from torch_choice.data import ChoiceDataset, utils
from torch_choice.model import ConditionalLogitModel

from torch_choice.utils.run_helper_lightning import run as run_lightning

/Users/tianyudu/miniforge3/envs/dev/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/tianyudu/miniforge3/envs/dev/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <B3E58761-2785-34C6-A89B-F37110C88A05> /Users/tianyudu/miniforge3/envs/dev/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <AE6DCE26-A528-35ED-BB3D-88890D27E6B9> /Users/tianyudu/miniforge3/envs/dev/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


# Load Dataset and Create the Model

In [2]:
# TODO: move this to a separate file.
def load_mode_canada_dataset():
    df = pd.read_csv('./public_datasets/ModeCanada.csv')
    df = df.query('noalt == 4').reset_index(drop=True)
    df.sort_values(by='case', inplace=True)
    item_index = df[df['choice'] == 1].sort_values(by='case')['alt'].reset_index(drop=True)
    item_names = ['air', 'bus', 'car', 'train']
    num_items = 4
    encoder = dict(zip(item_names, range(num_items)))
    print(f"{encoder=:}")
    item_index = item_index.map(lambda x: encoder[x])
    item_index = torch.LongTensor(item_index)
    print(f"{item_index=:}")
    price_cost_freq_ovt = utils.pivot3d(df, dim0='case', dim1='alt', values=['cost', 'freq', 'ovt'])
    print(f'{price_cost_freq_ovt.shape=:}')

    price_ivt = utils.pivot3d(df, dim0='case', dim1='alt', values='ivt')
    print(f'{price_ivt.shape=:}')
    
    session_income = df.groupby('case')['income'].first()
    session_income = torch.Tensor(session_income.values).view(-1, 1)
    dataset = ChoiceDataset(item_index=item_index,
                        price_cost_freq_ovt=price_cost_freq_ovt,
                        session_income=session_income,
                        price_ivt=price_ivt)
    return dataset

In [3]:
dataset = load_mode_canada_dataset()
dataset

encoder={'air': 0, 'bus': 1, 'car': 2, 'train': 3}
item_index=tensor([0, 0, 0,  ..., 2, 2, 2])
price_cost_freq_ovt.shape=torch.Size([2779, 4, 3])
price_ivt.shape=torch.Size([2779, 4, 1])
No `session_index` is provided, assume each choice instance is in its own session.


ChoiceDataset(label=[], item_index=[2779], user_index=[], session_index=[2779], item_availability=[], price_cost_freq_ovt=[2779, 4, 3], session_income=[2779, 1], price_ivt=[2779, 4, 1], device=cpu)

# Option 1: Use the `run()` Helper Function as Before

In [4]:
model = ConditionalLogitModel(coef_variation_dict={'price_cost_freq_ovt': 'constant',
                                                   'session_income': 'item',
                                                   'price_ivt': 'item-full',
                                                   'intercept': 'item'},
                              num_param_dict={'price_cost_freq_ovt': 3,
                                              'session_income': 1,
                                              'price_ivt': 1,
                                              'intercept': 1},
                              num_items=4)
DEVICE = "mps"
trained_model = run_lightning(model.to(DEVICE), dataset.to(DEVICE), learning_rate=0.3, num_epochs=5000, device=DEVICE)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


==================== model received ====================
ConditionalLogitModel(
  (coef_dict): ModuleDict(
    (price_cost_freq_ovt[constant]): Coefficient(variation=constant, num_items=4, num_users=None, num_params=3, 3 trainable parameters in total, device=mps:0).
    (session_income[item]): Coefficient(variation=item, num_items=4, num_users=None, num_params=1, 3 trainable parameters in total, device=mps:0).
    (price_ivt[item-full]): Coefficient(variation=item-full, num_items=4, num_users=None, num_params=1, 4 trainable parameters in total, device=mps:0).
    (intercept[item]): Coefficient(variation=item, num_items=4, num_users=None, num_params=1, 3 trainable parameters in total, device=mps:0).
  )
)
Conditional logistic discrete choice model, expects input features:

X[price_cost_freq_ovt[constant]] with 3 parameters, with constant level variation.
X[session_income[item]] with 1 parameters, with item level variation.
X[price_ivt[item-full]] with 1 parameters, with item-full level 

/Users/tianyudu/miniforge3/envs/dev/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name  | Type                  | Params
------------------------------------------------
0 | model | ConditionalLogitModel | 13    
------------------------------------------------
13        Trainable params
0         Non-trainable params
13        Total params
0.000     Total estimated model params size (MB)
/Users/tianyudu/miniforge3/envs/dev/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/tianyudu/miniforge3/en

Epoch 4999: 100%|██████████| 1/1 [00:00<00:00, 44.68it/s, loss=1.88e+03, v_num=11, train_loss=1.88e+3]

`Trainer.fit` stopped: `max_epochs=5000` reached.


Epoch 4999: 100%|██████████| 1/1 [00:00<00:00, 41.91it/s, loss=1.88e+03, v_num=11, train_loss=1.88e+3]
Time taken for training: 126.8951518535614
Skip testing, no test dataset is provided.
==================== model results ====================
Final Log-likelihood: [Training] -1880.45166015625, [Validation] N/A, [Test] N/A

Coefficients:

| Coefficient                     |   Estimation |   Std. Err. |
|:--------------------------------|-------------:|------------:|
| price_cost_freq_ovt[constant]_0 |  -0.0482738  |  0.00714503 |
| price_cost_freq_ovt[constant]_1 |   0.0962115  |  0.00511947 |
| price_cost_freq_ovt[constant]_2 |  -0.0441688  |  0.00325923 |
| session_income[item]_0          |  -0.101072   |  0.0189843  |
| session_income[item]_1          |  -0.0237517  |  0.00379524 |
| session_income[item]_2          |  -0.0348146  |  0.00402362 |
| price_ivt[item-full]_0          |   0.0587401  |  0.0101634  |
| price_ivt[item-full]_1          |  -0.014254   |  0.00614856 |
| price_

# Option 2: Have Full Control over the Training Loop